In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

%matplotlib inline

In [2]:
def inspect_object(ob):
    print(type(ob))
    print(ob)

Let's create a dataset, which is a 2D array. We will create it so that first part of the value is the row number and the second part is the column number

In [3]:
arr = np.array(list(range(1000))).reshape(100,10)
arr.shape

(100, 10)

In [4]:
arr

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
       [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19],
       [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29],
       [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39],
       [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49],
       [ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69],
       [ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89],
       [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99],
       [100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
       [110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
       [120, 121, 122, 123, 124, 125, 126, 127, 128, 129],
       [130, 131, 132, 133, 134, 135, 136, 137, 138, 139],
       [140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
       [150, 151, 152, 153, 154, 155, 156, 157, 158, 159],
       [160, 161, 162, 163, 164, 165, 166, 167, 168, 169

To Abdo: Check the following arguments in the documentation for dataset
 - window
 - batch
 - map and flat_map
 - prefetch size
 - buffer size for shuffling
 
 The following code is for a seq to seq model of a time series. We predict **multiple steps** in the future based on **multiple steps** in the past. We do the following:
 - window the dataset by **total** needed length, so we window by **past + future** size
 - **we do a small trick: we use the batch() function to transform a window to a dataset!**
 - Now that we have the dataset batched, we split it into past and future using map
 - we  batch with the desired mini batch size for training (To Abdo: What is the effect of having a small batch size vs a large batch size?)
 - To Abdo:What is the difference between from_tensor and from_tensor_slices? (I don't know, you'll have to explain it to me!!!)

In [14]:
# definitions
history_window = 3 # number of steps in the past
pred_window = 1    # number of steps in the future
prefetch_size = 1  

ds_len = 100
buffer_size = ds_len

window_size = history_window+pred_window
batch_size = 3


In [22]:
#TODO
raw_ds = tf.data.Dataset.from_tensor_slices(arr)
inspect_object(raw_ds)

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>
<TensorSliceDataset shapes: (10,), types: tf.int64>


In [27]:
for t in raw_ds:
    print(t[:7])

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([10 11 12 13 14 15 16], shape=(7,), dtype=int64)
tf.Tensor([20 21 22 23 24 25 26], shape=(7,), dtype=int64)
tf.Tensor([30 31 32 33 34 35 36], shape=(7,), dtype=int64)
tf.Tensor([40 41 42 43 44 45 46], shape=(7,), dtype=int64)
tf.Tensor([50 51 52 53 54 55 56], shape=(7,), dtype=int64)
tf.Tensor([60 61 62 63 64 65 66], shape=(7,), dtype=int64)
tf.Tensor([70 71 72 73 74 75 76], shape=(7,), dtype=int64)
tf.Tensor([80 81 82 83 84 85 86], shape=(7,), dtype=int64)
tf.Tensor([90 91 92 93 94 95 96], shape=(7,), dtype=int64)
tf.Tensor([100 101 102 103 104 105 106], shape=(7,), dtype=int64)
tf.Tensor([110 111 112 113 114 115 116], shape=(7,), dtype=int64)
tf.Tensor([120 121 122 123 124 125 126], shape=(7,), dtype=int64)
tf.Tensor([130 131 132 133 134 135 136], shape=(7,), dtype=int64)
tf.Tensor([140 141 142 143 144 145 146], shape=(7,), dtype=int64)
tf.Tensor([150 151 152 153 154 155 156], shape=(7,), dtype=int64)
tf.Tensor([160 161 16

In [23]:
window_ds = raw_ds.window(window_size, shift=1, drop_remainder=True)
inspect_object(window_ds)

<class 'tensorflow.python.data.ops.dataset_ops.WindowDataset'>
<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorShape([]))>


In [30]:
for t in window_ds:
    print(t)

<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset shapes: (10,), types: tf.int64>
<_VariantDataset sha

In [29]:
#Throws an error if we try to subscript it
for t in window_ds:
    print(t[:5])

TypeError: '_VariantDataset' object is not subscriptable

In [17]:
flat_map_ds = window_ds.flat_map(lambda w: w.batch(window_size))
inspect_object(flat_map_ds)

<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<FlatMapDataset shapes: (None, 10), types: tf.int64>


In [31]:
for t in flat_map_ds:
    print(t)

tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]], shape=(4, 10), dtype=int64)
tf.Tensor(
[[10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]], shape=(4, 10), dtype=int64)
tf.Tensor(
[[20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]], shape=(4, 10), dtype=int64)
tf.Tensor(
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]], shape=(4, 10), dtype=int64)
tf.Tensor(
[[40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]], shape=(4, 10), dtype=int64)
tf.Tensor(
[[50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87

In [18]:
map_ds = flat_map_ds.map(lambda example: (example[:history_window], example[history_window:]))
inspect_object(map_ds)

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
<MapDataset shapes: ((None, 10), (None, 10)), types: (tf.int64, tf.int64)>


In [19]:
batched_ds = map_ds.batch(batch_size).prefetch(prefetch_size)
inspect_object(batched_ds)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
<PrefetchDataset shapes: ((None, None, 10), (None, None, 10)), types: (tf.int64, tf.int64)>


In [21]:
for i, (past, future) in enumerate(batched_ds):
    print("Batch", i)
    print('past', past)
    print('future', future)
    print("==============")

Batch 0
past tf.Tensor(
[[[ 0  1  2  3  4  5  6  7  8  9]
  [10 11 12 13 14 15 16 17 18 19]
  [20 21 22 23 24 25 26 27 28 29]]

 [[10 11 12 13 14 15 16 17 18 19]
  [20 21 22 23 24 25 26 27 28 29]
  [30 31 32 33 34 35 36 37 38 39]]

 [[20 21 22 23 24 25 26 27 28 29]
  [30 31 32 33 34 35 36 37 38 39]
  [40 41 42 43 44 45 46 47 48 49]]], shape=(3, 3, 10), dtype=int64)
future tf.Tensor(
[[[30 31 32 33 34 35 36 37 38 39]]

 [[40 41 42 43 44 45 46 47 48 49]]

 [[50 51 52 53 54 55 56 57 58 59]]], shape=(3, 1, 10), dtype=int64)
Batch 1
past tf.Tensor(
[[[30 31 32 33 34 35 36 37 38 39]
  [40 41 42 43 44 45 46 47 48 49]
  [50 51 52 53 54 55 56 57 58 59]]

 [[40 41 42 43 44 45 46 47 48 49]
  [50 51 52 53 54 55 56 57 58 59]
  [60 61 62 63 64 65 66 67 68 69]]

 [[50 51 52 53 54 55 56 57 58 59]
  [60 61 62 63 64 65 66 67 68 69]
  [70 71 72 73 74 75 76 77 78 79]]], shape=(3, 3, 10), dtype=int64)
future tf.Tensor(
[[[60 61 62 63 64 65 66 67 68 69]]

 [[70 71 72 73 74 75 76 77 78 79]]

 [[80 81 82 83 8

# FULL CODE:

In [14]:

ds = tf.data.Dataset.range(ds_len)

#the output of window is a dataset of windows where each window
# is a dataset (a dataset of datasets of tensors, not tensors)
ds = ds.window(window_size, shift=1, drop_remainder=True)

#call a batch on each window. Bacth converts a window into a tensor.
ds = ds.flat_map(lambda w: w.batch(window_size))


ds = ds.map(lambda example: (example[:history_window], example[history_window:]))
#ds = ds.shuffle(buffer_size)
ds = ds.batch(batch_size).prefetch(prefetch_size)

for i, (past, future) in enumerate(ds):
    print("Batch", i)
    print('past', past)
    print('future', future)
    print("==============")

Batch 0
past tf.Tensor(
[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]], shape=(3, 5), dtype=int64)
future tf.Tensor(
[[5 6 7]
 [6 7 8]
 [7 8 9]], shape=(3, 3), dtype=int64)
Batch 1
past tf.Tensor(
[[3 4 5 6 7]
 [4 5 6 7 8]
 [5 6 7 8 9]], shape=(3, 5), dtype=int64)
future tf.Tensor(
[[ 8  9 10]
 [ 9 10 11]
 [10 11 12]], shape=(3, 3), dtype=int64)
Batch 2
past tf.Tensor(
[[ 6  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 12]], shape=(3, 5), dtype=int64)
future tf.Tensor(
[[11 12 13]
 [12 13 14]
 [13 14 15]], shape=(3, 3), dtype=int64)
Batch 3
past tf.Tensor(
[[ 9 10 11 12 13]
 [10 11 12 13 14]
 [11 12 13 14 15]], shape=(3, 5), dtype=int64)
future tf.Tensor(
[[14 15 16]
 [15 16 17]
 [16 17 18]], shape=(3, 3), dtype=int64)
Batch 4
past tf.Tensor(
[[12 13 14 15 16]
 [13 14 15 16 17]
 [14 15 16 17 18]], shape=(3, 5), dtype=int64)
future tf.Tensor(
[[17 18 19]
 [18 19 20]
 [19 20 21]], shape=(3, 3), dtype=int64)
Batch 5
past tf.Tensor(
[[15 16 17 18 19]
 [16 17 18 19 20]
 [17 18 19 20 21]], shape=(3,